Dans ce script, nous allons tester différents modèles de type convolution Nelle pour examiner leur performance. Nous allons ensuite tester s'ils sont robustes lorsqu'on applique une rotation à l'image d'entrée.

In [1]:
import retinoto_py as fovea

args = fovea.Params(batch_size=1, do_mask=False)
args

Welcome on macOS-26.1-arm64-arm-64bit-Mach-O	- Timestamp (UTC)  2025-12-04_21-47-31	 user laurent	Running on MPS device (Apple Silicon/MacOS)	 - macos_version = 26.1	 with device mps, pytorch==2.9.1
Random seed 2018 has been set.


Params(image_size=224, do_mask=False, do_fovea=False, rs_min=0.0, rs_max=-6.0, padding_mode='zeros', seed=2018, batch_size=1, num_workers=1, in_memory=False, model_name='convnext_base', num_epochs=24, n_train_stop=20480, n_val_stop=5120, lr=1e-06, delta1=0.1, delta2=0.007, weight_decay=0.03, label_smoothing=0.2, shuffle=True, verbose=True)

# testing each network on the validation dataset

In [2]:
import torch
VAL_DATA_DIR = args.DATAROOT / 'Imagenet_full' / 'val'

In [3]:
idx_to_label = fovea.get_idx_to_label(args)
idx_to_label[0]

Loading labels from local cache cached_data/imagenet_class_index.json...


'tench'

In [4]:
val_dataset = fovea.get_dataset(args, VAL_DATA_DIR, n_stop=args.n_val_stop)
val_loader = fovea.get_loader(args, val_dataset)

TypeError: exceptions must derive from BaseException

In [5]:
# Load the Pre-trained ResNet Model ---
import torchvision.models as models
# We'll use ResNet50, a powerful and common choice.
# `pretrained=True` downloads the model weights trained on ImageNet.
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
# Move the model to the selected device (GPU or CPU)
model = model.to(args.device)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
# args.model_name = 'convnext_tiny'
# model = fovea.load_model(args)
# model

In [7]:
# loading in one line
model = fovea.load_model(args)

# Set the model to evaluation mode.
model.eval()

# Make sure to load a model trained on the same number of classes
num_classes = len(val_loader.dataset.classes)
num_ftrs = model.fc.out_features
print(f'Model has {num_ftrs} output features to final FC layer for {num_classes} classes.')

NameError: name 'val_loader' is not defined

In [8]:
# The dataset provides a mapping from class index to class name (folder name)
class_to_idx = val_dataset.class_to_idx
# We often want the inverse mapping for printing results
idx_to_class = {v: k for k, v in class_to_idx.items()}

# --- 4. Evaluation Loop ---
correct_predictions = 0
total_predictions = 0

print(f"Starting evaluation on {len(val_loader.dataset)} images...")
print("-" * 50)

# We use vea. to create a nice progress bar
for images, true_labels in fovea.tqdm(val_loader, desc="Evaluating"):
    # Move data to the correct device
    images = images.to(args.device)
    true_labels = true_labels.to(args.device)

    # Get predictions (no need for gradients)
    with torch.no_grad():
        outputs = model(images)
        _, predicted_labels = torch.max(outputs, 1)

    # --- Check for Correctness ---
    # The batch size is 1, so we get the first element of the tensors
    predicted_label_idx = predicted_labels[0].item()
    true_label_idx = true_labels[0].item()

    # Get the human-readable names (folder names)
    predicted_class_name = idx_to_class[predicted_label_idx]
    true_class_name = idx_to_class[true_label_idx]

    # Check if the prediction was correct
    is_correct = (predicted_label_idx == true_label_idx)
    if is_correct:
        correct_predictions += 1
    total_predictions += 1

    # Get the image filename for printing
    # The DataLoader doesn't easily give filenames, so we can't print them without more work.
    # For this example, we'll just print the class names.
    status = "✅ Correct" if is_correct else "❌ Incorrect"
    if torch.rand(1) < .0001: print(f"True: {true_class_name:<20} ({idx_to_label[true_label_idx]}) | Predicted: {predicted_class_name:<20} (({idx_to_label[predicted_label_idx]})) | {status}")
        # # Get the human-readable label from our mapping
        # predicted_label = idx2label[predicted_index]

print("-" * 50)
accuracy = 100 * correct_predictions / total_predictions
print(f"Evaluation complete.")
print(f"Accuracy: {correct_predictions}/{total_predictions} ({accuracy:.2f}%)")

NameError: name 'val_dataset' is not defined